In [14]:
import json

log_file_path = 'round1.log'

json_entries = []

def try_parse_json(current_json):
    try:
        json_object = json.loads(current_json)
        json_entries.append(json_object)
        return True
    except json.JSONDecodeError:
        return False

# Read the file
with open(log_file_path, 'r') as file:
    lines = file.readlines()

current_json = ""
for line in lines:
    stripped_line = line.strip()  # Strip whitespace and new lines
    if stripped_line.startswith('{') and current_json:
        # Try to parse the current JSON before resetting if it's not empty
        if not try_parse_json(current_json):
            print(f"Failed to decode JSON: {current_json}")
        current_json = stripped_line  # Start a new JSON object
    else:
        current_json += stripped_line  # Continue adding to the current JSON string
    
    # Try to parse whenever you add anything potentially completing a JSON object
    if stripped_line.endswith('}'):
        if try_parse_json(current_json):
            current_json = ""  # Reset current JSON string if successfully parsed

# Final attempt to parse any remaining JSON data
if current_json and not try_parse_json(current_json):
    print(f"Failed to decode JSON at the end of the file: {current_json}")

# Display parsed JSON entries
for entry in json_entries:
    print(entry)





Failed to decode JSON: Sandbox logs:
Failed to decode JSON: Activities log:day;timestamp;product;bid_price_1;bid_volume_1;bid_price_2;bid_volume_2;bid_price_3;bid_volume_3;ask_price_1;ask_volume_1;ask_price_2;ask_volume_2;ask_price_3;ask_volume_3;mid_price;profit_and_loss1;0;AMETHYSTS;9998;9;9995;23;;;10005;23;;;;;10001.5;0.01;0;STARFRUIT;5047;23;;;;;5054;23;;;;;5050.5;0.01;100;STARFRUIT;5053;1;5050;2;5047;22;5054;22;;;;;5053.5;-71.31251;100;AMETHYSTS;10002;1;10000;2;9998;4;10004;1;10005;21;;;10003.0;0.01;200;AMETHYSTS;9996;1;9995;21;;;10004;1;10005;21;;;10000.0;2.01;200;STARFRUIT;5048;1;5047;21;;;5054;22;;;;;5051.0;-79.05468751;300;AMETHYSTS;9995;26;;;;;10005;26;;;;;10000.0;2.01;300;STARFRUIT;5053;1;5047;26;;;5054;26;;;;;5053.5;-78.18751;400;AMETHYSTS;9998;8;9996;2;9995;26;10004;2;10005;26;;;10001.0;2.01;400;STARFRUIT;5047;28;;;;;5054;28;;;;;5050.5;-65.1406251;500;STARFRUIT;5048;24;;;;;5053;3;5055;24;;;5050.5;-89.56251;500;AMETHYSTS;9998;8;9996;1;9995;23;10004;1;10005;23;;;10001.0;2.0

Failed to decode JSON: {"timestamp": 245900,"buyer": "","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5022,"quantity": 1},
Failed to decode JSON: {"timestamp": 246000,"buyer": "","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5023,"quantity": 1},
Failed to decode JSON: {"timestamp": 246000,"buyer": "SUBMISSION","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5020,"quantity": 2},
Failed to decode JSON: {"timestamp": 246100,"buyer": "SUBMISSION","seller": "","symbol": "AMETHYSTS","currency": "SEASHELLS","price": 9998,"quantity": 5},
Failed to decode JSON: {"timestamp": 246200,"buyer": "","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5018,"quantity": 15},
Failed to decode JSON: {"timestamp": 246400,"buyer": "","seller": "SUBMISSION","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5023,"quantity": 1},
Failed to decode JSON: {"timestamp": 246500,"buyer": "SUBMISSION","seller": "","symbol": "AMETHYSTS

Failed to decode JSON: {"timestamp": 516700,"buyer": "SUBMISSION","seller": "","symbol": "AMETHYSTS","currency": "SEASHELLS","price": 9998,"quantity": 1},
Failed to decode JSON: {"timestamp": 516700,"buyer": "","seller": "","symbol": "AMETHYSTS","currency": "SEASHELLS","price": 9998,"quantity": 1},
Failed to decode JSON: {"timestamp": 516800,"buyer": "","seller": "SUBMISSION","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5012,"quantity": 4},
Failed to decode JSON: {"timestamp": 516900,"buyer": "","seller": "SUBMISSION","symbol": "AMETHYSTS","currency": "SEASHELLS","price": 10002,"quantity": 6},
Failed to decode JSON: {"timestamp": 517000,"buyer": "SUBMISSION","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5011,"quantity": 5},
Failed to decode JSON: {"timestamp": 517600,"buyer": "","seller": "SUBMISSION","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5013,"quantity": 1},
Failed to decode JSON: {"timestamp": 517800,"buyer": "","seller": "","symbol": 

Failed to decode JSON: {"timestamp": 788800,"buyer": "","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5027,"quantity": 1},
Failed to decode JSON: {"timestamp": 789100,"buyer": "","seller": "","symbol": "AMETHYSTS","currency": "SEASHELLS","price": 9996,"quantity": 1},
Failed to decode JSON: {"timestamp": 789100,"buyer": "","seller": "SUBMISSION","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5026,"quantity": 8},
Failed to decode JSON: {"timestamp": 789100,"buyer": "","seller": "","symbol": "AMETHYSTS","currency": "SEASHELLS","price": 9996,"quantity": 7},
Failed to decode JSON: {"timestamp": 789300,"buyer": "","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5021,"quantity": 1},
Failed to decode JSON: {"timestamp": 789400,"buyer": "","seller": "","symbol": "STARFRUIT","currency": "SEASHELLS","price": 5026,"quantity": 2},
Failed to decode JSON: {"timestamp": 789400,"buyer": "","seller": "","symbol": "AMETHYSTS","currency": "SEASHELLS","pric

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
for entry in json_entries:
    lambda_log = entry.get('lambdaLog', '')
    # Parse out further details as necessary, e.g., prices, actions
    print(lambda_log)


In [8]:
# Example of tracking prices and actions
prices = []
actions = []
for entry in json_entries:
    if 'Acceptable buy price for AMETHYSTS :' in entry['lambdaLog']:
        parts = entry['lambdaLog'].split('\\n')
        for part in parts:
            if 'Acceptable buy price for AMETHYSTS :' in part:
                buy_price = float(part.split(':')[1].strip())
                prices.append(buy_price)
            if 'SELL AMETHYSTS at' in part:
                action = part.split('SELL')[1].strip()
                actions.append(action)

print('Prices:', prices)
print('Actions:', actions)



Prices: []
Actions: []
